In [ ]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import numpy as np

### Load video clip for inference

In [ ]:
image_dims = (100, 100)

In [ ]:
# Get the video path
sample_path = os.path.relpath('<Relative path to video sample>')

# Get video from path
cap = cv2.VideoCapture(sample_path)

frames16 = []
X = [] # Input samples list

counter = 0

while(cap.isOpened()) and counter != 161:
    # Store one sample (of 16 frames)
    if counter % 16 == 0 and counter != 0:
        X.append(frames16)
        # Reset frames16
        frames16 = []

    ret, frame = cap.read()
    # Check if a frame is capture
    if ret:
        # Resize the frame to image_dims x 3
        frame = cv2.resize(frame, image_dims, interpolation=cv2.INTER_CUBIC)
        # Store the frame in a list
        frames16.append(frame)
    else:
        break
    counter += 1

# Release everything when job is finished
cap.release()
cv2.destroyAllWindows()

In [ ]:
X = np.array(X, dtype=np.float32)

In [ ]:
# Max normalize input
X = X / 255

In [ ]:
X.shape

(10, 16, 100, 100, 3)

### Model

In [ ]:
lstm_units = 256
num_of_classes = 2

In [ ]:
class CNN_LSTM(tf.keras.Model):
    def __init__(self, lstm_units, num_of_classes):
        super(CNN_LSTM, self).__init__()
        self.lstm_units = lstm_units
        # Initialize inceptionV3
        self.inceptionV3 = tf.keras.applications.InceptionV3(
            input_shape=(image_dims[0], image_dims[1], 3), include_top=False, weights='imagenet', pooling='avg')
        self.inceptionV3.trainable = False
        
        # Define LSTM
        self.lstm_1 = tf.keras.layers.LSTM(
            lstm_units, 
            return_state=True,
            recurrent_initializer='glorot_uniform')
        
        self.fc = tf.keras.layers.Dense(num_of_classes)

    def call(self, x, hidden_state, cell_state):
        # x (batch size, 16, image_dims[0], image_dims[1], 3)
        # hidden_state (batch size, lstm_units)
        # cell_state (batch size, lstm_units)
        for i in range(0, x.shape[1]):
            # x[:, i, :, :, :] # (batch size, image_dims[0], image_dims[1], 3)
            out = self.inceptionV3(x[:, i, :, :, :]) # (batch size, 1, 2048)
            out = tf.expand_dims(out, 1)
            _, hidden_state, cell_state = self.lstm_1(
                out, initial_state=[hidden_state, cell_state])
            

        # Pass the last hidden state
        output = self.fc(hidden_state) # (batch size, num_of_classes)

        return output
  
    def initialize_state(self, batch_size):
        return tf.zeros((batch_size, self.lstm_units)), tf.zeros((batch_size, self.lstm_units))

In [ ]:
cnn_lstm = CNN_LSTM(lstm_units, num_of_classes)

In [ ]:
# Load model weights
cnn_lstm.load_weights('<Path to model weights>')

### Evaluate

In [ ]:
def evaluate(x):
    # Get intial hidden state and cell state
    hidden_state, cell_state = cnn_lstm.initialize_state(x.shape[0])
    # Predict
    predictions = cnn_lstm(x, hidden_state, cell_state)
    return predictions

In [ ]:
predictions = evaluate(X)

#### Take average of 10 predictions

In [ ]:
final_output = tf.reduce_sum(tf.argmax(predictions, axis=1)) / 10

#### Apply thresholding

In [ ]:
if final_output >= 0.5:
    print('Final output is Cheating')
else:
    print('Final output is Not Cheating')

Final output is Not Cheating
